<a href="https://colab.research.google.com/github/tassos96/Coursera_Capstone/blob/main/CapstoneProjectWeek3_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Question 1


##Import data

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#After retreiving the URL and creating a Beautiful soup object*
wikipedia = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wikipedia.content, 'lxml')

#Create a list
table_contents = []

#Find table and table data
table = soup.find_all('table')[0]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#Load table contents to a dataframe
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#Print dataframe
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [2]:
#Print dataframe shape
print(df.shape)

(103, 3)


#Question 2

##Import and install geocoder

In [3]:
!pip install geocoder
import geocoder

##The below suggested approach does not run.

In [4]:
# # initialize your variable to None
# lat_lng_coords = None

# postal_code = 'M3A'

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

##Alternative approach as suggested in the assignment

In [5]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
final_data = df.join(data.set_index('Postal Code'), on = 'Postal Code', how='inner') 
final_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [7]:
#Print final data shape
print(final_data.shape)

(103, 5)


#Question 3

##Based on lab from NYC neighbourhood we have to follow the same method

In [8]:
#Install and import Nominatim
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [9]:
#Find Toronto's coordinates
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [10]:
#Import folium
import folium

In [11]:
#Visualise on map
torontoMap = folium.Map(location=[latitude, longitude], zoom_start=11)


# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(torontoMap)  
    
torontoMap

In [12]:
#Initialize Foursquare API credentials
CLIENT_ID = '5DJBSQOGV4XVB34W1YJEUWREADLVPS0JP5IJKYZGDZYQHZPV'
CLIENT_SECRET = 'NBY5JXAM224OZEZSPPMOXKT4TYJH5NSN5LSMVNR3FSFVFRCG' 
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5DJBSQOGV4XVB34W1YJEUWREADLVPS0JP5IJKYZGDZYQHZPV
CLIENT_SECRET:NBY5JXAM224OZEZSPPMOXKT4TYJH5NSN5LSMVNR3FSFVFRCG


In [13]:
#Define a method to collect all nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
#Gather oronto venues
torontoVenues = getNearbyVenues(final_data['Neighbourhood'], final_data['Latitude'], final_data['Longitude'])

In [15]:
#Display first venues
torontoVenues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena


In [16]:
#Group venues by neighbourhood and find the unique
torontoVenues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
Neighbourhood,,,,
Agincourt,4,4,4,4
"Alderwood, Long Branch",8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18
Bayview Village,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22
...,...,...,...,...
Willowdale West,5,5,5,5
"Willowdale, Newtonbrook",2,2,2,2
Woburn,3,3,3,3


In [17]:
#Find unique venues per neighbourhood
print('There are {} uniques categories.'.format(len(torontoVenues['Venue Category'].unique())))

There are 224 uniques categories.


In [18]:
#Create one hot encoding
toronto_onehot = pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = torontoVenues['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
#Group by neighbourhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.055556,0.0,0.0,0.0,0.0,...,0.0,0.055556,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0


In [20]:
#Define a method to get most common venues in descending order
def getMostCommonVenues(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)
    
  return row_categories_sorted.index.values[0:num_top_venues]



In [21]:
#Import numpy
import numpy as np

In [22]:
#Take the top 10 venues. We will use them for clustering
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for i in np.arange(toronto_grouped.shape[0]):
  neighborhoods_venues_sorted.iloc[i, 1:] = getMostCommonVenues(toronto_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Discount Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Playground,Coffee Shop,Pub,Sandwich Place,Athletics & Sports,Discount Store,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Frozen Yogurt Shop,Park,Diner,Sandwich Place,Shopping Mall,Bridal Shop,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Greek Restaurant,Thai Restaurant,Pharmacy,Pizza Place,Cupcake Shop,Pub,Café


##Clustering

In [23]:
#Import k-means cluster
from sklearn.cluster import KMeans

In [24]:
# set number of clusters
clustersNum = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clustersNum, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [25]:
kmeans.labels_

array([4, 0, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4,
       3, 4, 4, 4, 4, 0, 4, 4, 4, 4, 1, 2, 4, 4, 4, 4, 4, 4, 3, 4, 4, 0,
       1, 4, 4, 2, 1, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 2, 4, 2, 4, 4,
       4, 0, 4, 0, 4, 4, 0, 2, 4, 4, 2], dtype=int32)

In [26]:
#Add clustering label column to most common venues
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Discount Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
1,0,"Alderwood, Long Branch",Pizza Place,Pharmacy,Playground,Coffee Shop,Pub,Sandwich Place,Athletics & Sports,Discount Store,Dessert Shop,Dim Sum Restaurant
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Frozen Yogurt Shop,Park,Diner,Sandwich Place,Shopping Mall,Bridal Shop,Sushi Restaurant
3,4,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
4,4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Greek Restaurant,Thai Restaurant,Pharmacy,Pizza Place,Cupcake Shop,Pub,Café
...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,Willowdale West,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Supermarket,Yoga Studio,Dim Sum Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
95,2,"Willowdale, Newtonbrook",Park,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
96,4,Woburn,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
97,4,Woodbine Heights,Intersection,Spa,Construction & Landscaping,Beer Store,Park,Athletics & Sports,Diner,Ethiopian Restaurant,Escape Room,Electronics Store


In [27]:
#Combine final_data with neighborhoods_venues_sorted
toronto_merged = final_data
toronto_merged.reset_index()
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Construction & Landscaping,Park,Food & Drink Shop,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Bakery,Pub,Park,Chocolate Shop,Performing Arts Venue,Dessert Shop,Restaurant,Café,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Boutique,Coffee Shop,Vietnamese Restaurant,Miscellaneous Shop,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Bank,Persian Restaurant,College Auditorium,Diner,Restaurant,Café


In [28]:
#drop nan
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])

In [29]:
#import libraries for visualization
import matplotlib.cm as cm
import matplotlib.colors as colors

In [30]:
#Visualize clusters on map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clustersNum)
ys = [i + x + (i*x)**2 for i in range(clustersNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

##Examine Clusters

In [31]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
8,East York,0.0,Pizza Place,Flea Market,Café,Intersection,Athletics & Sports,Gastropub,Gym / Fitness Center,Pharmacy,Bank,Eastern European Restaurant
17,Etobicoke,0.0,Pizza Place,Café,Pharmacy,Liquor Store,Beer Store,Coffee Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
19,East Toronto,0.0,Playground,Health Food Store,Pub,Neighborhood,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
32,Scarborough,0.0,Playground,Grocery Store,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
63,York,0.0,Pizza Place,Grocery Store,Convenience Store,Brewery,Bus Line,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
70,Etobicoke,0.0,Pizza Place,Sandwich Place,Playground,Coffee Shop,Middle Eastern Restaurant,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
72,North York,0.0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Supermarket,Yoga Studio,Dim Sum Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
77,Etobicoke,0.0,Sandwich Place,Park,Bus Line,Pizza Place,Construction & Landscaping,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
82,Scarborough,0.0,Fast Food Restaurant,Pizza Place,Thai Restaurant,Coffee Shop,Fried Chicken Joint,Italian Restaurant,Chinese Restaurant,Gas Station,Bank,Pharmacy


In [32]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Fast Food Restaurant,Construction & Landscaping,Park,Food & Drink Shop,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
6,Scarborough,1.0,Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
27,North York,1.0,Mediterranean Restaurant,Fast Food Restaurant,Pool,Golf Course,Dog Run,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
62,Central Toronto,1.0,Garden,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [33]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,2.0,Park,Women's Store,Pool,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
35,East York/East Toronto,2.0,Intersection,Convenience Store,Park,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
40,North York,2.0,Airport,Park,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
52,North York,2.0,Park,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
66,North York,2.0,Park,Convenience Store,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
85,Scarborough,2.0,Playground,Park,Intersection,Concert Hall,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
91,Downtown Toronto,2.0,Park,Playground,Trail,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
98,Etobicoke,2.0,Pool,Park,River,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop


In [34]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
101,Etobicoke,3.0,Pool,Baseball Field,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore


In [35]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4.0,Coffee Shop,Bakery,Pub,Park,Chocolate Shop,Performing Arts Venue,Dessert Shop,Restaurant,Café,French Restaurant
3,North York,4.0,Clothing Store,Boutique,Coffee Shop,Vietnamese Restaurant,Miscellaneous Shop,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store
4,Queen's Park,4.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Bank,Persian Restaurant,College Auditorium,Diner,Restaurant,Café
7,North York,4.0,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
9,Downtown Toronto,4.0,Café,Japanese Restaurant,Middle Eastern Restaurant,Coffee Shop,Clothing Store,Theater,Tea Room,Bakery,Ramen Restaurant,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,4.0,Coffee Shop,Bakery,Pub,Italian Restaurant,Restaurant,Pizza Place,Café,Japanese Restaurant,Caribbean Restaurant,Taiwanese Restaurant
97,Downtown Toronto,4.0,Café,Restaurant,Coffee Shop,Deli / Bodega,Gym,Tea Room,Beer Bar,Bakery,Steakhouse,Japanese Restaurant
99,Downtown Toronto,4.0,Sushi Restaurant,Gym,Japanese Restaurant,Dance Studio,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Sandwich Place,Restaurant
100,East Toronto Business,4.0,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Recording Studio
